## First we need to load some things

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from Shape_Maker import Data_Generator, make_rectangle, make_circle
from Callbacks.Visualizing_Model_Utils import TensorBoardImage

In [ ]:
%matplotlib notebook

In [ ]:
image_size = 64

## The make_rectangle and make_circle will both return circles and rectangles, and the Data_Generator will randomly create circles or rectangles

In [ ]:
plt.imshow(make_rectangle(image_size))

In [ ]:
plt.imshow(make_circle(image_size))

## What do we need? We need a way to generate larges amounts of training data for our model..

### This is a 'generator', it is something which continously creates data

In [ ]:
train_generator = Data_Generator(image_size=image_size,batch_size=64, num_examples_per_epoch=100)

In [ ]:
x,y = train_generator.__getitem__(0)
print(x.shape)
print(y.shape)

### X will be out data set, this will have a size [64 , 50, 50, 1]
This is 64 examples, of a 50x50 image, with 1 channel
### Y is our output class, we have two classes: box and not box, so a shape of [64, 2]

## Now lets make our network!

In [ ]:
from keras import Sequential
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Activation
from keras.optimizers import Adam

In [ ]:
model = Sequential([
    Conv2D(4, (3,3), input_shape=(image_size, image_size, 1), padding='same',name='Conv_0'), # Make 4 kernels,
    MaxPool2D((image_size)), # Pool into a 1x1x6 image
    Flatten(),
    Dense(2),
    Activation('softmax')
])

In [ ]:
model.compile(Adam(lr=1e-1), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit_generator(train_generator,epochs=10)

In [ ]:
layer_names = [i.name for i in model.layers]

In [ ]:
weights = model.layers[layer_names.index('Conv_0')].get_weights()[0][:, :, 0, :]

In [ ]:
conv_names = ['Conv_0']
weights = model.layers[layer_names.index(conv_names[0])].get_weights()[0][:, :, 0, :]
n_features = weights.shape[-1]
split = 2
while n_features / split % 2 == 0 and n_features / split >= split:
    split *= 2
split /= 2
images_per_row = int(n_features // split)
if len(weights.shape) == 4:
    rows_size = weights.shape[1]
    cols_size = weights.shape[2]
else:
    rows_size = weights.shape[0]
    cols_size = weights.shape[1]
n_cols = n_features // images_per_row
out_image = np.ones((rows_size * images_per_row + images_per_row - 1, n_cols * cols_size + n_cols - 1))*np.min(weights)
step = 0
for col in range(n_cols):
    for row in range(images_per_row):
        weight = weights[...,step]
        weight = (weight-np.mean(weight))/np.std(weight)
        out_image[row + row * rows_size:row + (row + 1) * rows_size,
        col + col * cols_size:col + (col + 1) * cols_size] = weight
        step += 1

In [ ]:
out_image.shape

In [ ]:
out_image.shape

In [ ]:
plt.imshow(out_image,cmap='gray')

In [ ]:
for i in range(1,weights.shape[-1]+1):
  plt.subplot(2,2,i)
  plt.imshow(x1w[:,:,i-1],interpolation="nearest",cmap="gray")
plt.show()

In [ ]:
def determine_accuracy(model, image_size= 64, num_examples=1000):
    truth = np.zeros((num_examples,1))
    guess = np.zeros((num_examples,1))
    index = 0
    for _ in range(num_examples//2):
        pred = model.predict(make_rectangle(image_size)[None,...,None])
        guess[index] = np.argmax(pred)
        truth[index] = 1
        index += 1
    for _ in range(num_examples//2):
        pred = model.predict(make_circle(image_size)[None,...,None])
        guess[index] = np.argmax(pred)
        index += 1
    print('Accuracy is ' + str((guess==truth).sum()/num_examples))

In [ ]:
determine_accuracy(model, image_size=image_size)

## What if we have no idea how many features are needed?
### We can increase the number of kernels, maybe it's easier to say 'this isn't a circle' than 'this is a rectangle'

In [ ]:
model = Sequential([
    Conv2D(16, (3,3), input_shape=(image_size, image_size, 1), padding='same'), # Make 4 kernels,
    MaxPool2D((image_size)), # Pool into a 1x1x6 image
    Flatten(),
    Dense(2),
    Activation('softmax')
])
model_deep.compile(Adam(lr=1e-3), loss='categorical_crossentropy', metrics=['accuracy'])
model_deep.fit_generator(train_generator,epochs=50, steps_per_epoch=100)

In [ ]:
determine_accuracy(model_deep)